In [1]:
import pandas as pd
data = pd.read_csv('Training(2000-2017).csv')

In [2]:
training_data = data.drop(['Player', 'School', 'Pos'], axis=1)
training_data = training_data.dropna()
training_data.head()

,Ht,Wt,40yd,Vertical,Bench,Broad_Jump,3Cone,Shuttle,Draft_Pick
0,76,334,5.56,25.0,23.0,94.0,8.48,4.98,188
1,78,312,5.34,28.0,20.0,96.0,7.72,4.73,247
3,76,302,5.07,31.5,17.0,103.0,7.76,4.58,206
4,77,297,5.16,31.0,28.0,100.0,7.82,4.48,112
5,79,304,5.08,31.0,28.0,103.0,7.65,4.65,123


In [3]:
cond = training_data['Draft_Pick'] == -1
training_data.loc[cond,'Draft_Pick'] = 'Not Drafted'
cond = training_data['Draft_Pick'] != 'Not Drafted'
training_data.loc[cond,'Draft_Pick'] = 'Drafted'
training_data.head()

,Ht,Wt,40yd,Vertical,Bench,Broad_Jump,3Cone,Shuttle,Draft_Pick
0,76,334,5.56,25.0,23.0,94.0,8.48,4.98,Drafted
1,78,312,5.34,28.0,20.0,96.0,7.72,4.73,Drafted
3,76,302,5.07,31.5,17.0,103.0,7.76,4.58,Drafted
4,77,297,5.16,31.0,28.0,100.0,7.82,4.48,Drafted
5,79,304,5.08,31.0,28.0,103.0,7.65,4.65,Drafted


In [4]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot

training_features = training_data.drop(['Draft_Pick'], axis=1)
training_target = training_data['Draft_Pick']

def evaluate_model(model, X, y):
    x_train, x_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)
    model.fit(x_train,y_train)
    scores = model.score(x_test,y_test)
    return scores

models = dict()
for p in [0.0, 0.0001, 0.001, 0.01, 0.1, 1.0]:
    key = '%.4f' % p
    if p == 0.0:
        models[key] = LogisticRegression(solver='lbfgs', penalty='none', class_weight = { '0:0.33', '1:0.67'}, max_iter = 400)
    else:
        models[key] = LogisticRegression(solver='lbfgs', penalty='l2', C=p, class_weight = { '0:0.33', '1:0.67' }, max_iter = 400)
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, training_features, training_target)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

>0.0000 0.790 (0.000)
>0.0001 0.714 (0.000)
>0.0010 0.647 (0.000)
>0.0100 0.681 (0.000)
>0.1000 0.748 (0.000)
>1.0000 0.723 (0.000)


In [5]:
import pandas as pd
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot
data = pd.read_csv('Testing(2018-2021).csv')
data2 = pd.read_csv('Training(2000-2017).csv')

In [6]:
testing_data = data.drop(['Player', 'School', 'Pos'], axis=1)
testing_data = testing_data.dropna()

training_data = data2.drop(['Player', 'School', 'Pos'], axis=1)
training_data = testing_data.dropna()

In [7]:
cond = testing_data['Draft_Pick'] != -1
testing_data.loc[cond,'Draft_Pick'] = 1

cond = training_data['Draft_Pick'] != -1
training_data.loc[cond,'Draft_Pick'] = 1

actuals = testing_data['Draft_Pick']
predict = testing_data.drop(['Draft_Pick'],axis=1)

training_features = training_data.drop(['Draft_Pick'], axis=1)
training_target = training_data['Draft_Pick']
training_target = training_target.astype('int')

training_data['Draft_Pick'].describe()

count    124.000000
mean       0.225806
std        0.978124
min       -1.000000
25%       -1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: Draft_Pick, dtype: float64

In [16]:
lrh = LogisticRegression(solver='lbfgs', penalty='none', class_weight = { '0:0.33', '1:0.67' }, max_iter = 375)

In [17]:
lrh.fit(training_features,training_target)

LogisticRegression(class_weight={'1:0.67', '0:0.33'}, max_iter=375,
                   penalty='none')

In [18]:
predictions = lrh.predict(predict)
originals = actuals.to_numpy()

In [19]:
cnt = 0
right = 0
while cnt<predictions.size:
    if predictions[cnt] == originals[cnt]:
        right = right + 1
    cnt = cnt + 1
print(right/cnt)

0.7016129032258065
